# What are your chances of winning the US Open?
Suppose you’re playing a match at the U.S. Open, and you’re slightly better than the competition: your chances of winning any given point are exactly 55 percent. (Yes, most players are more likely to win the points they serve, but we’re simplifying things a bit.) What are your chances of winning a three-set match, as played by the women, or a five-set match, as played by the men? And what are your chances of winning the whole tournament (seven consecutive matches)?

https://fivethirtyeight.com/features/what-are-your-chances-of-winning-the-u-s-open/

## 1. Game
### a. Normal
Need to have at least 4 points and be at least 2 points ahead.
### b. Tie-breaker (for a set)
If the score for the set is 6-6, first to reach 7 points with 2 points surplus wins

## 2. Set
### a. Normal
Need to win at least 6 games with a difference of 2
### b. Tie-breaker
If it gets to 6-6, 1.b applies

## 3. Match
For women 2 sets win the game, for men, 3.

## 1a. Normal Game
I will start with the odds of winning a normal game, e.g. the first or second game of a match.

This problem has to do with the conjunction of the following two:
- The winner has at least 4 points (eg. 15, 30, 40, win)
- The winner scores at least 2 more points than the loser.

### Initial thoughts:
- This is kind of a random walk problem with an added condition of at least 4 successes
- Once this is solved 1b is easy as it's the same problem with different arguments

# Mathematical Solution
## Ways of winning a normal game:
4-0: $(0.55)^4$

4-1: $4(0.45)^1(0.55)^4$

4-2: $10(0.45)^2(0.55)^4$

3-3: $20(0.45)^3(0.55)^3$ times the probability of coming out ahead by 2 points.

After 3-3, we start a game of random walk which ends with either player reaching 2 points.

$$0.55^2 + 0.55^2 (2 \times 0.45 \times 0.55) + 0.55^2 (2 \times 0.45 \times 0.55)^2 + \dots$$

$$= \sum_{i = 0}^\infty 0.55^2  (2 \times 0.45 \times 0.55)^i$$

$$= \frac{0.55^2}{1 - 2 \times 0.45 \times 0.55}$$

$$\approx 0.6$$

In [8]:
rand_walk_to_get_ahead_by_two = (.55**2)/(1-2*.45*.55)
fourzero = (0.55)**4
fourone = 4*(.45)*(.55)**4
fourtwo = 10*((.45)**2)*((.55)**4)
threethree = 20*(.45)**3*(.55)**3*rand_walk_to_get_ahead_by_two
print(fourzero, fourone, fourtwo, threethree)
win_game_prob = fourzero + fourone + fourtwo + threethree
print('Probability of winning a normal game: '+str(win_game_prob))

0.09150625000000003 0.16471125000000006 0.18530015625000007 0.18163084622524764
Probability of winning a normal game: 0.6231485024752478


## What about a tie-breaker game?

7-0: $(0.55)^7$

7-1: $7(0.55)^7(0.45)$

7-2: ${8 \choose 2}(0.55)^7(0.45)^2$

7-3: ${9 \choose 3}(0.55)^7(0.45)^3$

7-4: ${10 \choose 4}(0.55)^7(0.45)^4$

7-5: ${11 \choose 5}(0.55)^7(0.45)^5$

6-6: ${12 \choose 6}(0.55)^6(0.45)^6 \times 0.6$


In [9]:
sevenzero = ((.55)**7)
sevenone = 7*((.45)**1)*((.55)**7)
seventwo = 28*((.45)**2)*((.55)**7)
seventhree = 84*((.45)**3)*((.55)**7)
sevenfour = 210*((.45)**4)*((.55)**7)
sevenfive = 462*((.45)**5)*((.55)**7)
sixsix = 924*((.45)**6)*((.55)**6)*rand_walk_to_get_ahead_by_two
win_tie_break_prob = sevenzero + sevenone + seventwo + seventhree + sevenfour + sevenfive + sixsix

print('Probability of winning a tie-breaker: '+str(win_tie_break_prob))

Probability of winning a tie-breaker: 0.6541507671793694


## Set

(6-0) through (6-4) are similar.

7-5: This requires that the score be 5-5 at some point, and the better player win the next two games.

6-6: This requires that the score be 5-5 at some point. times winning the tie-breaker.

In [10]:
sixzero = win_game_prob**6
sixone = 6*win_game_prob**6*(1-win_game_prob)
sixtwo = 21*win_game_prob**6*(1-win_game_prob)**2
sixthree = 56*win_game_prob**6*(1-win_game_prob)**3                    
sixfour = 126*win_game_prob**6*(1-win_game_prob)**4
sevenfive = 252*win_game_prob**7*(1-win_game_prob)**5
sixsix = 504*win_game_prob**6*(1-win_game_prob)**6*win_tie_break_prob
win_set_prob = sixzero + sixone + sixtwo + sixthree + sixfour + sevenfive + sixsix
print('Probability of winning a set: '+str(win_set_prob))

Probability of winning a set: 0.8150419926646904


## Match

Probability of winning 2 or 3 sets for women and men, respectively.

In [11]:
win_match_prob_women = win_set_prob**2 + 2*win_set_prob**2*(1-win_set_prob)
win_match_prob_men = win_set_prob**3 + 3*win_set_prob**3*(1-win_set_prob) + 6*win_set_prob**3*(1-win_set_prob)**2
print('Probability of winning a match (women): '+str(win_match_prob_women))
print('Probability of winning a match (men): '+str(win_match_prob_men))

Probability of winning a match (women): 0.9100262353311684
Probability of winning a match (men): 0.9529824438604558


## Tournament

In [12]:
womens = win_match_prob_women**7
mens = win_match_prob_men**7
print('Probability of winning a tournament (women): '+str(womens))
print('Probability of winning a tournament (men): '+str(mens))

Probability of winning a tournament (women): 0.5168653160428079
Probability of winning a tournament (men): 0.7138291847643412


# Simulation

In [1]:
import numpy as np

def Game(tie_break=False):
    #Simulates a single game. Returns 1 if the better player wins, 0, otherwise.
    apoint, bpoint = 0, 0
    min_required_points = 4
    
    #Is it a tie-breaker game
    if tie_break == True:
        min_required_points = 7
        
    while abs(apoint - bpoint) < 2 or max(apoint,bpoint) < min_required_points:
        point = np.random.choice(2,1, p=[0.45,0.55])
        if point == 1:
            apoint += 1
        else:
            bpoint += 1
    #print('g: '+str(apoint)+' ' + str(bpoint))
    if apoint > bpoint:
        return 1
    return 0

In [2]:
def Set():
    #Simulates a single set. Returns 1 if the better player wins, 0, otherwise.
    agame, bgame = 0, 0
    while abs(agame - bgame) < 2 or max(agame, bgame) < 6:
        game_result = Game()
        if game_result == 1:
            agame += 1
        else:
            bgame += 1
        
        #Check to see if a tie-breaker game is needed
        if agame == 6 and bgame == 6:
            game_result = Game(tie_break = True)
            if game_result == 1:
                agame += 1
            else:
                bgame += 1
            break
            
    #print('s: '+str(agame)+' ' + str(bgame))
    if agame > bgame:
        return 1
    return 0
            

In [3]:
def Match(male=False):
    #Simulates a single match. Returns 1 if the better player wins, 0, otherwise.
    aset, bset = 0, 0
    required_set_count = 2
    
    if male == True:
        required_set_count = 3
        
    while aset < required_set_count and bset < required_set_count:
        set_result = Set()
        if set_result == 1:
            aset += 1
        else:
            bset += 1
    #print('m: '+str(aset)+' ' + str(bset))
    if aset > bset:
        return 1
    return 0

In [4]:
def MatchTest(ran,male=False):
    #Runs function 'Match' 'ran' times and returns a's win ratio
    awin = 0
    for i in range(ran):
        awin += Match(male)
    return awin/ran

In [7]:
#Simulation offemale matches
MatchTest(10000)

0.9092

In [13]:
#Simulation of male matches
MatchTest(10000, male=True)

0.9502

# Remarks
* Simulation seems to confirm the mathematical reuslts, which is good.
* I'm pretty confident that I haven't missed or overcounted the ways in which a game or set could be won, but you never know!
* I'm curious to see if there are more elegant solutions.
* I could have made the combinatorial task a little less tedious by writing a function in python, so there would be less clutter. But I preferred to go manual on my first try to be able to see more clearly what gets counted and what doesn't.


# Useful info:

* Info on random walks: https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-042j-mathematics-for-computer-science-fall-2010/readings/MIT6_042JF10_chap20.pdf

* How to count the ways in which a game could turn out given a specific score: https://en.wikipedia.org/wiki/Stars_and_bars_(combinatorics)